In [4]:
import numpy as np
import pandas as pd
import math 

from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import PolynomialFeatures, normalize
from doubleml import DoubleMLData, DoubleMLPLR

import warnings
warnings.filterwarnings('ignore')

import os
os.cpu_count()


8

# DML with 25 covariate model 

#### By Shahin Karami

# Importing Data 

In [5]:
#importing data.
#First converted the data to a CSV via Stata so i can inport

df = pd.read_excel('sle_merged_altered.xlsx')
df.head(10)
dfcopy = df.copy()

collist = list(dfcopy.columns)

# Y variable 

In [6]:
# dependent var that we are looking into 

y = dfcopy[["student_knowledge"]] # are we sure we want to use "student_knowledge" and not "student_proficiency"?
#y

# 25 + District/Region Effects Nuisence Parameters (X vars)

In [7]:
# Nuisense Parameters... These are all from the "Variable name" column in the codebook, except for light_GDP

X_list = ["absence_rate", "student_attendance", "students_enrolled", "content_proficiency","ecd_student_proficiency",
          "infrastructure","teach_score","operational_management","instructional_leadership","principal_knowledge_score",
          "principal_management", "teacher_attraction","teacher_selection_deployment","teacher_support",
          "teaching_evaluation","teacher_monitoring","intrinsic_motivation", "standards_monitoring", "sch_monitoring",
          "sch_management_clarity", "sch_management_attraction", "sch_selection_deployment", "sch_support", 
          "principal_evaluation","light_GDP"]
X = dfcopy[X_list]

X.shape
X



,absence_rate,student_attendance,students_enrolled,content_proficiency,ecd_student_proficiency,infrastructure,teach_score,operational_management,instructional_leadership,principal_knowledge_score,...,teacher_monitoring,intrinsic_motivation,standards_monitoring,sch_monitoring,sch_management_clarity,sch_management_attraction,sch_selection_deployment,sch_support,principal_evaluation,light_GDP
0,50.000000,42.307692,257.0,0.000000,0.000000,0.571429,2.666667,4.25,2.000000,3.0,...,3.000000,3.276267,5.00,3.357143,5.0,NaN,5.0,4.25,5.000000,0.012253
1,25.000000,32.812500,457.0,0.000000,0.000000,1.857143,2.638889,4.75,4.000000,3.0,...,1.400000,3.823413,5.00,3.657143,5.0,5.000,5.0,4.25,5.000000,0.165421
2,66.666667,46.153846,131.0,0.000000,0.000000,0.571429,2.041667,3.50,2.000000,3.0,...,2.000000,3.435467,5.00,3.657143,5.0,4.335,5.0,4.25,5.000000,0.004514
3,71.428571,100.000000,418.0,33.333333,33.333333,3.857143,2.152778,4.25,4.333333,3.0,...,2.333333,2.760267,5.00,3.657143,5.0,3.000,4.0,4.25,5.000000,0.011286
4,0.000000,39.130435,214.0,0.000000,0.000000,2.714286,2.666667,3.50,5.000000,3.0,...,2.000000,3.614400,5.00,2.914286,5.0,NaN,4.0,4.25,4.666667,0.013866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,33.333333,63.888889,217.0,20.000000,100.000000,2.571429,2.680556,4.25,4.800000,4.0,...,1.800000,4.100747,5.00,3.657143,5.0,2.500,5.0,1.00,5.000000,5.844281
298,72.727273,25.000000,200.0,0.000000,100.000000,2.285714,2.694444,NaN,5.000000,3.0,...,2.666667,3.311467,5.00,3.557143,5.0,1.000,5.0,4.25,4.666667,4.218267
299,NaN,100.000000,NaN,0.000000,0.000000,NaN,NaN,NaN,5.000000,5.0,...,2.500000,4.653333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.026119
300,75.000000,NaN,506.0,0.000000,33.333333,NaN,NaN,4.00,3.666667,5.0,...,3.000000,3.904800,2.84,2.485714,1.0,NaN,4.0,1.00,5.000000,0.309883


# Policy of Intrest (D Vars) Quasi-Arithmetic Mean with Weighted Aggregation

In [8]:
BE_alt = dfcopy[["bureaucratic_efficiency_alt"]]
IDM_alt = dfcopy[["impartial_decision_making_alt"]]
MA_alt = dfcopy[["mandates_accountability_alt"]]
QOB_alt = dfcopy[["quality_bureaucracy_alt"]]
NLG_alt = dfcopy[["national_learning_goals_alt"]]

# Policy of Intrest (D Vars) Quasi-Arithmetic Mean

In [9]:
BE_quas = dfcopy[["bureaucratic_efficiency_quas"]]
IDM_quas = dfcopy[["impartial_decision_making_quas"]]
MA_quas = dfcopy[["mandates_accountability_quas"]]
QOB_quas = dfcopy[["quality_bureaucracy_quas"]]
NLG_quas = dfcopy[["national_learning_goals_quas"]]

# Policy of Intrest (D Vars) Arithmetic Mean with Weighted Aggregation

In [10]:
BE_cw = dfcopy[["bureaucratic_efficiency_cw"]]
IDM_cw = dfcopy[["impartial_decision_making_cw"]]
MA_cw = dfcopy[["mandates_accountability_cw"]]
QOB_cw = dfcopy[["quality_bureaucracy_cw"]]
NLG_cw = dfcopy[["national_learning_goals_cw"]]

# Policy of Intrest (D Vars) Arithmetic Mean

In [11]:
BE = dfcopy[["bureaucratic_efficiency"]]
IDM = dfcopy[["impartial_decision_making"]]
MA = dfcopy[["mandates_accountability"]]
QOB = dfcopy[["quality_bureaucracy"]]
NLG = dfcopy[["national_learning_goals"]]

# Combining y, D, X

In this chunk of code you can define which index you would like to choose. Namely you have the choice between the four indexes created above.

1. Arithmetic Mean: BE, IDM, MA, QOB, NLG 

2. Arithmetic Mean with Weighted Aggregation: BE_cw, IDM_cw, MA_cw, QOB_cw, NLG_cw

3. Quasi-Arithmetic: BE_quas, IDM_quas, MA_quas, QOB_quas, NLG_quas

4. Quasi-Arithmetic with Weighted Aggregation: BE_alt, IDM_alt, MA_alt, QOB_alt, NLG_alt

Once you define the index you want here you wont have to alter any other code chunk. 

In [13]:
# Combining all my columns into a single new dataframe
combined_df25 = pd.DataFrame()
combined_df25["Student Knowledge"] = y

# Make sure define the correct index you want
combined_df25["Bureaucratic Efficiency"] = BE_alt
combined_df25["Impartial Decision Making"] = IDM_alt
combined_df25["Mandates & Accountability"] = MA_alt
combined_df25["Quality of Bureaucracy"] = QOB_alt
combined_df25["National Learning Goals"] = NLG_alt
combined_df25 = pd.concat([combined_df25, X], axis=1)
combined_df25.describe()

,Student Knowledge,Bureaucratic Efficiency,Impartial Decision Making,Mandates & Accountability,Quality of Bureaucracy,National Learning Goals,absence_rate,student_attendance,students_enrolled,content_proficiency,...,teacher_monitoring,intrinsic_motivation,standards_monitoring,sch_monitoring,sch_management_clarity,sch_management_attraction,sch_selection_deployment,sch_support,principal_evaluation,light_GDP
count,298.000000,301.000000,301.000000,301.000000,301.000000,301.000000,300.000000,299.000000,299.000000,297.000000,...,298.000000,298.000000,299.000000,299.000000,300.000000,285.000000,299.000000,300.00000,300.000000,301.000000
mean,36.815271,2.866357,3.028776,3.299021,3.338033,2.433793,29.262864,64.128781,315.969900,9.941077,...,2.410850,3.760555,4.282408,2.929336,4.518333,3.544596,4.418060,3.95750,4.578889,0.849009
std,16.549401,0.245111,0.424943,0.500845,0.319894,0.340656,24.577456,25.914659,191.715592,20.191435,...,0.774665,0.491570,0.827697,0.738400,1.186813,1.242648,0.625902,0.98826,0.635912,2.421012
min,9.001401,2.476424,1.839485,2.288758,2.920013,1.870313,0.000000,5.785124,40.000000,0.000000,...,1.000000,2.493600,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,2.000000,0.000000
25%,23.745766,2.608049,2.826634,3.027657,2.992953,2.265449,0.000000,44.142857,200.000000,0.000000,...,2.000000,3.422522,3.960000,2.471429,5.000000,2.335000,4.000000,4.25000,4.666667,0.009674
50%,34.291769,2.916002,3.070236,3.257661,3.416200,2.455508,25.000000,63.636364,289.000000,0.000000,...,2.500000,3.816600,4.600000,3.285714,5.000000,4.335000,4.000000,4.25000,4.666667,0.020960
75%,48.952332,3.087671,3.344143,3.785335,3.532901,2.683739,50.000000,87.500000,385.500000,20.000000,...,3.000000,4.144542,5.000000,3.557143,5.000000,4.335000,5.000000,4.50000,5.000000,0.253130
max,92.380392,3.187921,3.527610,4.044951,4.161776,3.184945,100.000000,100.000000,2006.000000,100.000000,...,4.750000,4.939520,5.000000,3.900000,5.000000,5.000000,5.000000,4.75000,5.000000,26.278387


# Cleaning up NA's and Creating Higher Order and Interaction Variables

In [19]:
# Cleaning dataframe to drop NaN, Then split Nuisense param so I can run Polynomial feature
combined_clean = combined_df25.dropna()
combined_clean = combined_clean.reset_index(drop=True)
combined_clean.describe()

X_update = combined_clean.iloc[:,6:] # only selecting the covariates and not y or D

X_update = normalize(X_update) # we only normalize the covariates

X_update = pd.DataFrame(X_update, columns=X.columns)

poly = PolynomialFeatures(degree=3) # default is to include_bias = True in other words there will be a column of 1s added
X_poly = poly.fit_transform(X_update)
X_poly_df = pd.DataFrame(X_poly, columns=poly.get_feature_names(X.columns))

# pulling all the column names into a list for the DoubleMLData
X_col = list(X_poly_df.columns[1:]) # [1:] to remove the column of 1s 
#X_poly_df.columns[1:]
X_poly_df.describe()
X_poly_df.shape
X_poly_df

,1,absence_rate,student_attendance,students_enrolled,content_proficiency,ecd_student_proficiency,infrastructure,teach_score,operational_management,instructional_leadership,...,sch_support^3,sch_support^2 principal_evaluation,sch_support^2 light_GDP,sch_support principal_evaluation^2,sch_support principal_evaluation light_GDP,sch_support light_GDP^2,principal_evaluation^3,principal_evaluation^2 light_GDP,principal_evaluation light_GDP^2,light_GDP^3
0,1.0,0.054444,0.071458,0.995238,0.000000,0.000000,0.004044,0.005747,0.010344,0.008711,...,7.928655e-07,9.327829e-07,3.086047e-08,1.097392e-06,3.630643e-08,1.201173e-09,1.291049e-06,4.271345e-08,1.413144e-09,4.675289e-11
1,1.0,0.430261,0.297873,0.845463,0.000000,0.000000,0.003688,0.013177,0.022589,0.012908,...,2.063651e-05,2.427824e-05,2.192047e-08,2.856264e-05,2.578879e-08,2.328432e-11,3.360311e-05,3.033975e-08,2.739331e-11,2.473302e-14
2,1.0,0.162867,0.228014,0.953098,0.076005,0.076005,0.008795,0.004909,0.009691,0.009881,...,9.100201e-07,1.070612e-06,2.416599e-09,1.259543e-06,2.843058e-09,6.417388e-12,1.481816e-06,3.344774e-09,7.549868e-12,1.704166e-14
3,1.0,0.168116,0.224155,0.958264,0.000000,0.000000,0.005764,0.007425,0.013449,0.012609,...,2.918014e-06,3.432957e-06,8.413109e-09,4.038773e-06,9.897776e-09,2.425637e-11,4.751498e-06,1.164444e-08,2.853690e-11,6.993506e-14
4,1.0,0.130766,0.170606,0.957646,0.000000,0.188885,0.005397,0.004276,0.008028,0.003589,...,7.222234e-07,7.602352e-07,9.805788e-10,8.002476e-07,1.032188e-09,1.331354e-12,8.423659e-07,1.086514e-09,1.401425e-12,1.807609e-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,1.0,0.240872,0.481744,0.823783,0.000000,0.160581,0.002065,0.008163,0.016861,0.009635,...,8.582550e-06,9.423977e-06,1.953544e-08,1.034790e-05,2.145068e-08,4.446620e-11,1.136240e-05,2.355369e-08,4.882564e-11,1.012131e-13
228,1.0,0.239238,0.478477,0.842119,0.000000,0.000000,0.003418,0.010367,0.015550,0.016747,...,1.095425e-07,5.111985e-07,7.417825e-10,2.385593e-06,3.461652e-09,5.023084e-12,1.113277e-05,1.615438e-08,2.344106e-11,3.401451e-14
229,1.0,0.000000,0.551718,0.579304,0.220687,0.551718,0.008670,0.014712,0.016552,0.023172,...,1.799837e-05,1.641957e-05,3.665518e-08,1.497925e-05,3.343982e-08,7.465133e-11,1.366528e-05,3.050650e-08,6.810297e-11,1.520337e-13
230,1.0,0.201530,0.238654,0.801082,0.000000,0.503825,0.010796,0.011686,0.017634,0.021832,...,1.165406e-05,1.208569e-05,8.053514e-06,1.253331e-05,8.351792e-06,5.565363e-06,1.299751e-05,8.661119e-06,5.771488e-06,3.845932e-06


# Recombining y, D, X (Now without NA's)

In [20]:
# Combining all my columns into a single new dataframe
combined_clean_poly = pd.DataFrame()
combined_clean_poly["Student Knowledge"] = combined_clean["Student Knowledge"]
combined_clean_poly["Bureaucratic Efficiency"] = combined_clean["Bureaucratic Efficiency"]
combined_clean_poly["Impartial Decision Making"] = combined_clean["Impartial Decision Making"]
combined_clean_poly["Mandates & Accountability"] = combined_clean["Mandates & Accountability"]
combined_clean_poly["Quality of Bureaucracy"] = combined_clean["Quality of Bureaucracy"]
combined_clean_poly["National Learning Goals"] = combined_clean["National Learning Goals"]
combined_clean_poly = pd.concat([combined_clean_poly, X_poly_df], axis=1)

combined_clean_poly = combined_clean_poly.drop(columns="1") #removing the intercept column "1" created by PolynomialFeatures

combined_clean_poly.describe()
combined_clean_poly

,Student Knowledge,Bureaucratic Efficiency,Impartial Decision Making,Mandates & Accountability,Quality of Bureaucracy,National Learning Goals,absence_rate,student_attendance,students_enrolled,content_proficiency,...,sch_support^3,sch_support^2 principal_evaluation,sch_support^2 light_GDP,sch_support principal_evaluation^2,sch_support principal_evaluation light_GDP,sch_support light_GDP^2,principal_evaluation^3,principal_evaluation^2 light_GDP,principal_evaluation light_GDP^2,light_GDP^3
0,30.737745,2.575943,1.839485,3.258808,2.920013,3.184945,0.054444,0.071458,0.995238,0.000000,...,7.928655e-07,9.327829e-07,3.086047e-08,1.097392e-06,3.630643e-08,1.201173e-09,1.291049e-06,4.271345e-08,1.413144e-09,4.675289e-11
1,32.664760,2.575943,1.839485,3.258808,2.920013,3.184945,0.430261,0.297873,0.845463,0.000000,...,2.063651e-05,2.427824e-05,2.192047e-08,2.856264e-05,2.578879e-08,2.328432e-11,3.360311e-05,3.033975e-08,2.739331e-11,2.473302e-14
2,31.884532,2.575943,1.839485,3.258808,2.920013,3.184945,0.162867,0.228014,0.953098,0.076005,...,9.100201e-07,1.070612e-06,2.416599e-09,1.259543e-06,2.843058e-09,6.417388e-12,1.481816e-06,3.344774e-09,7.549868e-12,1.704166e-14
3,33.786648,2.575943,1.839485,3.258808,2.920013,3.184945,0.168116,0.224155,0.958264,0.000000,...,2.918014e-06,3.432957e-06,8.413109e-09,4.038773e-06,9.897776e-09,2.425637e-11,4.751498e-06,1.164444e-08,2.853690e-11,6.993506e-14
4,48.823268,2.575943,1.839485,3.258808,2.920013,3.184945,0.130766,0.170606,0.957646,0.000000,...,7.222234e-07,7.602352e-07,9.805788e-10,8.002476e-07,1.032188e-09,1.331354e-12,8.423659e-07,1.086514e-09,1.401425e-12,1.807609e-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,9.001401,2.841893,2.826634,3.184846,3.416200,2.690090,0.240872,0.481744,0.823783,0.000000,...,8.582550e-06,9.423977e-06,1.953544e-08,1.034790e-05,2.145068e-08,4.446620e-11,1.136240e-05,2.355369e-08,4.882564e-11,1.012131e-13
228,23.888889,2.841893,2.826634,3.184846,3.416200,2.690090,0.239238,0.478477,0.842119,0.000000,...,1.095425e-07,5.111985e-07,7.417825e-10,2.385593e-06,3.461652e-09,5.023084e-12,1.113277e-05,1.615438e-08,2.344106e-11,3.401451e-14
229,26.045752,2.841893,2.826634,3.184846,3.416200,2.690090,0.000000,0.551718,0.579304,0.220687,...,1.799837e-05,1.641957e-05,3.665518e-08,1.497925e-05,3.343982e-08,7.465133e-11,1.366528e-05,3.050650e-08,6.810297e-11,1.520337e-13
230,58.315178,2.698573,3.296723,3.257661,3.114314,2.110603,0.201530,0.238654,0.801082,0.000000,...,1.165406e-05,1.208569e-05,8.053514e-06,1.253331e-05,8.351792e-06,5.565363e-06,1.299751e-05,8.661119e-06,5.771488e-06,3.845932e-06


# Creating the DoubleMLData Objects for Each Policy Var (5 total)

To use the DoubleML package the user must first create a DoubleMLData object. In this Object you define what your y,d,x as well as the data you are going to be calling your model parameters from. DoubleMLData can also include a `z_cols` argument should the user want to include a intrumental variable to the analysis.

More information on the DoubleML package can be found for both the python and R APIs here: https://docs.doubleml.org/stable/index.html

In [21]:
# Bureaucratic Efficiency
dmlPLR_data_BE = DoubleMLData(
    combined_clean_poly,
    y_col='Student Knowledge',
    d_cols='Bureaucratic Efficiency',
    x_cols= X_col
)

# Impartial Desicion Making
dmlPLR_data_IDM = DoubleMLData(
    combined_clean_poly,
    y_col='Student Knowledge',
    d_cols='Impartial Decision Making',
    x_cols=X_col
)

# Mandates & Accountability
dmlPLR_data_MA = DoubleMLData(
    combined_clean_poly,
    y_col='Student Knowledge',
    d_cols='Mandates & Accountability',
    x_cols=X_col
)

# Quality of Bureaucracy
dmlPLR_data_QOB = DoubleMLData(
    combined_clean_poly,
    y_col='Student Knowledge',
    d_cols='Quality of Bureaucracy',
    x_cols=X_col
)

# National Learning Goals
dmlPLR_data_NLG = DoubleMLData(
    combined_clean_poly,
    y_col='Student Knowledge',
    d_cols='National Learning Goals',
    x_cols=X_col
)

# Partial Linear Regrssion DoubleML using 

Note that any Machine Learing model can be used in the DoubleML Package. We opt to use a linear model fitted by minimizing a regularized empirical loss with Stochastic Gradient Descent (SGD) from `sklearn`. 

Refer to the documentation for sklearn.linear_model.SGDRegressor for more information 

In [22]:
# Run Partial Linear Regrssion DoubleML with the various DoubleML datasets

# Bureaucratic Efficiency  
dml_plrSGDR_BE = DoubleMLPLR(
    obj_dml_data=dmlPLR_data_BE,
    ml_l=SGDRegressor(learning_rate = "optimal",alpha = 0.0001),
    ml_m=SGDRegressor(learning_rate = "optimal",alpha = 0.0001),
    n_folds=2
)

BE = dml_plrSGDR_BE.fit(n_jobs_cv = os.cpu_count()).bootstrap(method='normal', n_rep_boot=500000)

# Impartial Desicion Making
dml_plrSGDR_IDM = DoubleMLPLR(
    obj_dml_data=dmlPLR_data_IDM,
    ml_l=SGDRegressor(learning_rate = "optimal",alpha = 0.0001),
    ml_m=SGDRegressor(learning_rate = "optimal",alpha = 0.0001),
    n_folds=2
)

IDM = dml_plrSGDR_IDM.fit(n_jobs_cv = os.cpu_count()).bootstrap(method='normal', n_rep_boot=500000)

# Mandates & Accountability
dml_plrSGDR_MA = DoubleMLPLR(
    obj_dml_data=dmlPLR_data_MA,
    ml_l=SGDRegressor(learning_rate = "optimal",alpha = 0.0001),
    ml_m=SGDRegressor(learning_rate = "optimal",alpha = 0.0001),
    n_folds=2
)

MA = dml_plrSGDR_MA.fit(n_jobs_cv = os.cpu_count()).bootstrap(method='normal', n_rep_boot=500000)

# Quality of Bureaucracy
dml_plrSGDR_QOB = DoubleMLPLR(
    obj_dml_data=dmlPLR_data_QOB,
    ml_l=SGDRegressor(learning_rate = "optimal",alpha = 0.0001),
    ml_m=SGDRegressor(learning_rate = "optimal",alpha = 0.0001),
    n_folds=2
)

QOB = dml_plrSGDR_QOB.fit(n_jobs_cv = os.cpu_count()).bootstrap(method='normal', n_rep_boot=500000)

# National Learning Goals
dml_plrSGDR_NLG = DoubleMLPLR(
    obj_dml_data=dmlPLR_data_NLG,
    ml_l=SGDRegressor(learning_rate = "optimal",alpha = 0.0001),
    ml_m=SGDRegressor(learning_rate = "optimal",alpha = 0.0001),
    n_folds=2
)

NLG = dml_plrSGDR_NLG.fit(n_jobs_cv = os.cpu_count()).bootstrap(method='normal', n_rep_boot=500000)

dml_plr_SGDR = pd.concat([BE.summary, IDM.summary, MA.summary,QOB.summary, NLG.summary])
dml_plr_SGDR

# In order to save results from all 4 indicies I alter the name of the dml_plr_SGDR 
# to include the indicies within the variable name (see below for examples)
# due to time constraints I was not able to automate this process

,coef,std err,t,P>|t|,2.5 %,97.5 %
Bureaucratic Efficiency,0.708202,0.037462,18.904470,1.047877e-79,0.634777,0.781626
Impartial Decision Making,0.753275,0.029378,25.640983,5.329886e-145,0.695695,0.810854
Mandates & Accountability,0.834208,0.040663,20.515247,1.573671e-93,0.754510,0.913906
Quality of Bureaucracy,-0.448554,0.019971,-22.460682,1.006322e-111,-0.487696,-0.409412
National Learning Goals,0.856036,0.030310,28.242369,1.766209e-175,0.796629,0.915443


# SGDRegressor with Quasi-Arithmetic Mean with Weighted Aggregeation


In [17]:
dml_plr_SGDR_alt

,coef,std err,t,P>|t|,2.5 %,97.5 %
Bureaucratic Efficiency,-0.134032,0.020754,-6.458247,1.059228e-10,-0.174709,-0.093356
Impartial Decision Making,0.747803,0.054566,13.704486,9.544130e-43,0.640855,0.854751
Mandates & Accountability,-0.381462,0.024671,-15.462179,6.244942e-54,-0.429816,-0.333109
Quality of Bureaucracy,-0.868319,0.053058,-16.365444,3.375572e-60,-0.972311,-0.764327
National Learning Goals,-0.119921,0.131606,-0.911217,3.621810e-01,-0.377863,0.138021


# SGDRegressor with Quasi-Arithmetic Mean


In [32]:
dml_plr_SGDR_quas

,coef,std err,t,P>|t|,2.5 %,97.5 %
Bureaucratic Efficiency,-0.235000,0.018507,-12.698112,6.057346e-37,-0.271273,-0.198728
Impartial Decision Making,0.419727,0.017994,23.326213,2.403426e-120,0.384460,0.454994
Mandates & Accountability,0.144163,0.008276,17.420291,5.788155e-68,0.127944,0.160383
Quality of Bureaucracy,0.009618,0.001775,5.417828,6.032732e-08,0.006139,0.013098
National Learning Goals,0.509738,0.072413,7.039285,1.932283e-12,0.367811,0.651666


# SGDRegressor with Arithmetic Mean and Weighted aggregeation


In [68]:
 dml_plr_SGDR_cw

,coef,std err,t,P>|t|,2.5 %,97.5 %
Bureaucratic Efficiency,0.964283,0.051973,18.553643,7.620809e-77,0.862418,1.066148
Impartial Decision Making,0.525790,0.052264,10.060185,8.284277e-24,0.423353,0.628226
Mandates & Accountability,-0.948383,0.035140,-26.988947,1.992524e-160,-1.017255,-0.879510
Quality of Bureaucracy,-0.254879,0.030971,-8.229476,1.880347e-16,-0.315582,-0.194176
National Learning Goals,1.267730,0.151766,8.353191,6.644333e-17,0.970274,1.565186


# SGDRegressor with Arithmetic Mean 

In [51]:
dml_plr_SGDR_raw

,coef,std err,t,P>|t|,2.5 %,97.5 %
Bureaucratic Efficiency,0.580960,0.020923,27.766795,1.092549e-169,0.539952,0.621968
Impartial Decision Making,1.209707,0.100379,12.051421,1.906345e-33,1.012968,1.406445
Mandates & Accountability,0.536688,0.020896,25.683820,1.772450e-145,0.495732,0.577643
Quality of Bureaucracy,-0.223298,0.017752,-12.578855,2.759999e-36,-0.258090,-0.188505
National Learning Goals,0.465358,0.075839,6.136127,8.455764e-10,0.316716,0.614000
